# SELECT, FROM & WHERE

Today, we're going to learn how to use SELECT, FROM and WHERE to get data from a specific column based on the value of another column. For the purposes of this explanation, we'll be using this imaginary database, `pet_records` which has just one table in it, called `pets`, which looks like this:

![](https://i.imgur.com/Ef4Puo3.png)

### SELECT ... FROM
___

The most basic SQL query is to select a single column from a specific table. To do this, you need to tell SELECT which column to select and then specify what table that column is from using from. 

> **Do you need to capitalize SELECT and FROM?** No, SQL doesn't care about capitalization. However, it's customary to capitalize your SQL commands and it makes your queries a bit easier to read.

So, if we wanted to select the "Name" column from the pets table of the pet_records database (if that database were accessible as a BigQuery dataset on Kaggle , which it is not, because I made it up), we would do this:

    SELECT Name
    FROM `bigquery-public-data.pet_records.pets`

Which would return the highlighted data from this figure.

![](https://i.imgur.com/8FdVyFP.png)

### WHERE ...
___

When you're working with BigQuery datasets, you're almost always going to want to return only certain rows, usually based on the value of a different column. You can do this using the WHERE clause, which will only return the rows where the WHERE clause evaluates to true.

Let's look at an example:

    SELECT Name
    FROM `bigquery-public-data.pet_records.pets`
    WHERE Animal = "Cat"

This query will only return the entries from the "Name" column that are in rows where the "Animal" column has the text "Cat" in it. Those are the cells highlighted in blue in this figure:

![](https://i.imgur.com/Va52Qdl.png)


## Example: What are all the U.S. cities in the OpenAQ dataset?
___

Now that you've got the basics down, let's work through an example with a real dataset. Today we're going to be working with the OpenAQ dataset, which has information on air quality around the world. (The data in it should be current: it's updated weekly.)

To help get you situated, I'm going to run through a complete query first. Then it will be your turn to get started running your queries!

First, I'm going to set up everything we need to run queries and take a quick peek at what tables are in our database.

In [1]:
# import package with helper functions 
import bq_helper

# create a helper object for this dataset
open_aq = bq_helper.BigQueryHelper(active_project="bigquery-public-data",
                                   dataset_name="openaq")

# print all the tables in this dataset (there's only one!)
open_aq.list_tables()

['global_air_quality']

I'm going to take a peek at the first couple of rows to help me see what sort of data is in this dataset.

In [2]:
# print the first couple rows of the "global_air_quality" dataset
open_aq.head("global_air_quality")

,location,city,country,pollutant,value,timestamp,unit,source_name,latitude,longitude,averaged_over_in_hours
0,Mobile_Cle Elum,037,US,pm25,0.00,2017-09-26 20:00:00+00:00,µg/m³,AirNow,47.197630,-120.958230,1.0
1,Mobile_WhiteSalmon,039,US,pm25,0.00,2017-09-26 20:00:00+00:00,µg/m³,AirNow,45.732414,-121.492330,1.0
2,Mobile_Newport,051,US,pm25,0.00,2017-09-21 18:00:00+00:00,µg/m³,AirNow,48.186485,-117.049160,1.0
3,FR20047,Ain,FR,o3,2.13,2018-02-13 07:00:00+00:00,µg/m³,EEA France,45.823223,4.953958,1.0
4,FR20047,Ain,FR,pm10,18.70,2018-02-13 07:00:00+00:00,µg/m³,EEA France,45.823223,4.953958,1.0


Great, everything looks good! Now that I'm set up, I'm going to put together a query. I want to select all the values from the "city" column for the rows there the "country" column is "us" (for "United States"). 

> **What's up with the triple quotation marks (""")?** These tell Python that everything inside them is a single string, even though we have line breaks in it. The line breaks aren't necessary, but they do make it much easier to read your query.

In [4]:
# query to select all the items from the "city" column where the
# "country" column is "us"
query = """SELECT city
            FROM `bigquery-public-data.openaq.global_air_quality`
            WHERE country = 'US'
        """

> **Important:**  Note that the argument we pass to FROM is *not* in single or double quotation marks (' or "). It is in backticks (\`). If you use quotation marks instead of backticks, you'll get this error when you try to run the query: `Syntax error: Unexpected string literal` 

Now I can use this query to get information from our open_aq dataset. I'm using the `BigQueryHelper.query_to_pandas_safe()` method here because it won't run a query if it's larger than 1 gigabyte, which helps me avoid accidentally running a very large query. See the [Scavenger Hunt Handbook ](https://www.kaggle.com/rtatman/sql-scavenger-hunt-handbook/)for more details. 

In [5]:
# the query_to_pandas_safe will only return a result if it's less
# than one gigabyte (by default)
us_cities = open_aq.query_to_pandas_safe(query)

Now I've got a dataframe called us_cities, which I can use like I would any other dataframe:

In [6]:
# What five cities have the most measurements taken there?
us_cities.city.value_counts().head()

Phoenix-Mesa-Scottsdale                     85
Houston                                     75
Los Angeles-Long Beach-Santa Ana            60
New York-Northern New Jersey-Long Island    57
Riverside-San Bernardino-Ontario            56
Name: city, dtype: int64

# Scavenger hunt
___

Now it's your turn! Here's the questions I would like you to get the data to answer:

* Which countries use a unit other than ppm to measure any type of pollution? (Hint: to get rows where the value *isn't* something, use "!=")
* Which pollutants have a value of exactly 0?

In order to answer these questions, you can fork this notebook by hitting the blue "Fork Notebook" at the very top of this page (you may have to scroll up).  "Forking" something is making a copy of it that you can edit on your own without changing the original.

In [7]:
# Your code goes here :)
query1 = """SELECT country,pollutant
            FROM `bigquery-public-data.openaq.global_air_quality`
            WHERE pollutant != 'ppm'
        """
result1 = open_aq.query_to_pandas_safe(query1)
result1.tail(10)

,country,pollutant
17097,US,pm25
17098,VN,pm25
17099,VN,pm25
17100,VN,pm25
17101,XK,pm25
17102,ZA,so2
17103,ZA,so2
17104,ZA,so2
17105,ZA,pm10
17106,ZA,pm10


In [13]:
query2 = """SELECT country,pollutant,value
            FROM `bigquery-public-data.openaq.global_air_quality`
            WHERE value = 0
        """
result2 = open_aq.query_to_pandas_safe(query2)
result2.tail(10)

,country,pollutant,value
752,US,pm25,0.0
753,US,pm25,0.0
754,US,pm25,0.0
755,US,pm25,0.0
756,US,pm25,0.0
757,US,pm25,0.0
758,US,pm25,0.0
759,US,pm25,0.0
760,US,pm25,0.0
761,US,pm25,0.0


- Check whether the datasets has the air pollution situation of China (CH), and appreantly no. 

In [17]:
query3 = """SELECT DISTINCT country
            FROM `bigquery-public-data.openaq.global_air_quality`
        """
result3 = open_aq.query_to_pandas_safe(query3)
result3

,country
0,US
1,FR
2,TH
3,IN
4,NL
5,GB
6,CH
7,PT
8,ES
9,BR


- Check in which U.S. city has the worst pm2.5 air pullution? 

In [21]:
query4 = """SELECT city, pollutant, AVG(value) AS average_value
            FROM `bigquery-public-data.openaq.global_air_quality`
            WHERE country = 'US' AND pollutant = 'pm25'
            GROUP BY city,pollutant
            ORDER BY average_value DESC
        """
result4 = open_aq.query_to_pandas_safe(query4)
result4.head(10)

,city,pollutant,average_value
0,IDAHO,pm25,122.933333
1,CLACKAMAS,pm25,59.100000
2,Hanford-Corcoran,pm25,53.000000
3,SAN JOAQUIN,pm25,48.000000
4,SONOMA,pm25,44.000000
5,Fresno,pm25,32.000000
6,MT1,pm25,31.000000
7,Bangor,pm25,31.000000
8,CA8 - ARB,pm25,27.833333
9,Greeley,pm25,27.300000
